## Multi Model Endpoint

라이브러리

In [39]:
# 데이터 처리 및 분석
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', None)
import numpy as np

# 머신러닝
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# AWS 관련
import sagemaker
from sagemaker.utils import name_from_base
import boto3
import awswrangler as wr
from sagemaker.utils import name_from_base

# 시각화
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 기타 유틸리티
import pickle
import os
import json
import string
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from dotenv import load_dotenv
load_dotenv()

[nltk_data] Downloading package punkt to /Users/dante/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /Users/dante/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/dante/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

데이터 다운로드

In [2]:
df = pd.read_csv('https://github.com/mohitgupta-1O1/Kaggle-SMS-Spam-Collection-Dataset-/raw/master/spam.csv', encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


### 데이터 전처리

컬럼 생성 / 중복제거 / 결측치

In [3]:
df.shape

(5572, 5)

In [4]:
# 스팸 여부 컬럼 생성
df['isSpam'] = (df['v1'] == 'spam').astype(int)
# 메시지 컬럼 생성
df['message'] = df['v2']
df = df[['isSpam', 'message']]
df.head()

,isSpam,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
# 결측치 확인
df.isnull().sum()

isSpam     0
message    0
dtype: int64

In [6]:
# 중복 확인
df.duplicated().sum()

403

In [7]:
# 중복 제거
df = df.drop_duplicates(keep='first')

In [8]:
# 스팸 여부 비율 확인
df.isSpam.value_counts()

isSpam
0    4516
1     653
Name: count, dtype: int64

단어수, 글자수 컬럼 생성 및 분포 확인

In [11]:
# 글자수 컬럼 생성
df['num_chars'] = df['message'].apply(len)
# 단어수 컬럼 생성
df['num_words'] = df['message'].apply(lambda x:len(nltk.word_tokenize(x)))

In [12]:
# 서브플롯 생성
fig = make_subplots(rows=1, cols=2, subplot_titles=('글자 수 분포', '단어 수 분포'))

# 글자 수 히스토그램
fig.add_trace(
    go.Histogram(x=df[df['isSpam']==0]['num_chars'], name='정상', opacity=0.75),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=df[df['isSpam']==1]['num_chars'], name='스팸', opacity=0.75),
    row=1, col=1
)

# 단어 수 히스토그램
fig.add_trace(
    go.Histogram(x=df[df['isSpam']==0]['num_words'], name='정상', opacity=0.75),
    row=1, col=2
)
fig.add_trace(
    go.Histogram(x=df[df['isSpam']==1]['num_words'], name='스팸', opacity=0.75),
    row=1, col=2
)

# 레이아웃 업데이트
fig.update_layout(barmode='overlay', title='스팸 여부에 따른 글자 수와 단어 수 분포')
fig.update_xaxes(title_text='글자 수', row=1, col=1)
fig.update_xaxes(title_text='단어 수', row=1, col=2)
fig.update_yaxes(title_text='빈도', row=1, col=1)
fig.update_yaxes(title_text='빈도', row=1, col=2)

# 그래프 표시
fig.show()

불용어 / 구두점 제거

In [14]:
# 영어 불용어 목록 가져오기
stopwords = nltk.corpus.stopwords.words('english')
# Porter 스테머 초기화
ps = nltk.PorterStemmer()


def transform_text(text):
    # 텍스트를 소문자로 변환
    text = text.lower()
    # 텍스트를 단어로 토큰화
    text = nltk.word_tokenize(text)
    
    y = []
    # 알파벳과 숫자만 포함된 단어 선택
    for i in text:
        if i.isalnum():
            y.append(i)
    
    text = y[:]
    y.clear()
    
    # 불용어와 구두점 제거
    for i in text:
        if i not in stopwords and i not in string.punctuation:
            y.append(i)
            
    text = y[:]
    y.clear()
    
    # 단어 어간 추출
    for i in text:
        y.append(ps.stem(i))
    
    # 처리된 단어들을 공백으로 연결하여 문자열로 반환
    return " ".join(y)

In [15]:
df['prep_message'] = df['message'].apply(transform_text)

In [16]:
# 스팸 메시지의 단어들을 모아 spam_corpus 리스트 생성
spam_corpus = []
for msg in df[df['isSpam'] == 1]['prep_message'].tolist():
    for word in msg.split():
        spam_corpus.append(word)

# spam_corpus의 길이 출력 (스팸 메시지에 사용된 총 단어 수)
len(spam_corpus)

9939

In [17]:
# 햄(정상) 메시지의 단어들을 모아 ham_corpus 리스트 생성
ham_corpus = []
for msg in df[df['isSpam'] == 0]['prep_message'].tolist():
    for word in msg.split():
        ham_corpus.append(word)

# ham_corpus의 길이 출력 (정상 메시지에 사용된 총 단어 수)
len(ham_corpus)

35404

텍스트 벡터라이즈

In [18]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
cv = CountVectorizer()
tfidf = TfidfVectorizer(max_features=3000)

In [19]:
X = tfidf.fit_transform(df['prep_message']).toarray()
y = df['isSpam'].values
print('X.shape : ', X.shape)
print('y.shape : ', y.shape)

X.shape :  (5169, 3000)
y.shape :  (5169,)


In [24]:
os.makedirs('dataset/smsspam', exist_ok=True)
with open('dataset/smsspam/sms_spam_preprocessed_data.pkl', 'wb') as f:
    pickle.dump((X, y), f)

훈련 데이터 준비

In [25]:
with open('dataset/smsspam/sms_spam_preprocessed_data.pkl', 'rb') as f:
    X, y = pickle.load(f)
    
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size=0.1, random_state = 2024)
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size=0.3, random_state = 2024)

### 컨테이너 모델 훈련

경로 설정

In [26]:
bucket_name = 'dante-sagemaker'
project_name = 'smsspam'

s3_training_file_location = r's3://{0}/{1}/input/{2}/'.format(bucket_name, project_name, 'training')
s3_validation_file_location =r's3://{0}/{1}/input/{2}/'.format(bucket_name, project_name, 'validation')

s3_xgb_output_location = r's3://{0}/{1}/output/{2}'.format(bucket_name, project_name, 'xgboost')
s3_xgb_checkpoint_location = r's3://{0}/{1}/checkpoint/{2}'.format(bucket_name, project_name, 'xgboost')

print('s3_training_file_location : ', s3_training_file_location)
print('s3_validation_file_location : ', s3_validation_file_location)
print('s3_xgb_output_location : ', s3_xgb_output_location)
print('s3_xgb_checkpoint_location : ', s3_xgb_checkpoint_location)

s3_training_file_location :  s3://dante-sagemaker/smsspam/input/training/
s3_validation_file_location :  s3://dante-sagemaker/smsspam/input/validation/
s3_xgb_output_location :  s3://dante-sagemaker/smsspam/output/xgboost
s3_xgb_checkpoint_location :  s3://dante-sagemaker/smsspam/checkpoint/xgboost


SageMaker 세션 및 역할 설정

In [28]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')

데이터 S3 업로드

In [29]:
y_train_2d = y_train.reshape(-1, 1)
train_data = np.hstack((y_train_2d, X_train))

y_val_2d = y_val.reshape(-1, 1)
val_data = np.hstack((y_val_2d, X_val))

wr.s3.to_csv(df=pd.DataFrame(train_data), path=s3_training_file_location + 'train_data.csv', index=False, header=False, boto3_session=boto3_session)
wr.s3.to_csv(df=pd.DataFrame(val_data), path=s3_validation_file_location + 'val_data.csv', index=False, header=False, boto3_session=boto3_session)

{'paths': ['s3://dante-sagemaker/smsspam/input/validation/val_data.csv'],
 'partitions_values': {}}

모델훈련

In [31]:
# XGBoost 모델 훈련 함수
def train_xgb_model(**hyperparameters) :
    # XGBoost 모델 컨테이너 이미지 URI 가져오기
    model_container = sagemaker.image_uris.retrieve("xgboost",sagemaker_session.boto_region_name,version="1.7-1")
    # 작업 이름 생성
    base_job_name = project_name + '-' + 'xgb-train'
    
    # XGBoost 추정기 생성
    xgb_estimator = sagemaker.estimator.Estimator(
        model_container,
        role,
        input_mode='File',
        instance_count=1,
        instance_type='ml.m5.xlarge',
        output_path=s3_xgb_output_location,
        sagemaker_session=sagemaker_session,
        max_run=60 * 60,
        max_wait=60 * 60 * 2,
        use_spot_instances=True,
        base_job_name=base_job_name,
    )
    
    # 하이퍼파라미터 설정
    xgb_estimator.set_hyperparameters(
        **hyperparameters
    )

    # 훈련 데이터 입력 구성
    training_input_config = sagemaker.session.TrainingInput(
        s3_data=s3_training_file_location,
        content_type='text/csv')

    # 검증 데이터 입력 구성
    validation_input_config = sagemaker.session.TrainingInput(
        s3_data=s3_validation_file_location,
        content_type='text/csv')

    # 데이터 채널 설정
    data_channels = {'train': training_input_config, 'validation': validation_input_config}

    # 모델 훈련 실행
    xgb_estimator.fit(data_channels, wait=True, logs=True)
    return xgb_estimator


In [33]:
# 모델 A의 하이퍼파라미터 설정
modelA_hyperparameters = {
    'max_depth': 5,              # 트리의 최대 깊이
    'eta': 0.1,                  # 학습률
    'gamma': 4,                  # 리프 노드를 추가적으로 나누기 위한 최소 손실 감소
    'min_child_weight': 6,       # 자식 노드에 필요한 최소 가중치 합
    'subsample': 0.8,            # 각 트리마다 사용할 훈련 데이터의 비율
    'objective': 'binary:logistic', # 이진 분류를 위한 목적 함수
    'num_round': 200,            # 부스팅 라운드 수
    'early_stopping_rounds': 10, # 조기 종료를 위한 라운드 수
    'eval_metric': 'logloss'     # 평가 지표
}

# 모델 B의 하이퍼파라미터 설정
modelB_hyperparameters = {
    'max_depth': 7,              # 트리의 최대 깊이 (모델 A보다 깊음)
    'eta': 0.15,                 # 학습률 (모델 A보다 높음)
    'gamma': 4,                  # 리프 노드를 추가적으로 나누기 위한 최소 손실 감소
    'min_child_weight': 8,       # 자식 노드에 필요한 최소 가중치 합 (모델 A보다 높음)
    'subsample': 0.8,            # 각 트리마다 사용할 훈련 데이터의 비율
    'objective': 'binary:logistic', # 이진 분류를 위한 목적 함수
    'num_round': 200,            # 부스팅 라운드 수
    'early_stopping_rounds': 10, # 조기 종료를 위한 라운드 수
    'eval_metric': 'logloss'     # 평가 지표
}

In [34]:
estimator_A = train_xgb_model(**modelA_hyperparameters)

INFO:sagemaker:Creating training-job with name: smsspam-xgb-2024-08-24-23-31-19-942-2024-08-24-23-31-20-040


2024-08-24 23:31:20 Starting - Starting the training job......
2024-08-24 23:32:14 Downloading - Downloading input data...
2024-08-24 23:32:34 Downloading - Downloading the training image......
2024-08-24 23:33:43 Training - Training image download completed. Training in progress....
2024-08-24 23:34:19 Uploading - Uploading generated training model[2024-08-24 23:33:53.119 ip-10-0-219-22.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-08-24 23:33:53.140 ip-10-0-219-22.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-08-24:23:33:53:INFO] Imported framework sagemaker_xgboost_container.training
[2024-08-24:23:33:53:INFO] Failed to parse hyperparameter eval_metric value logloss to Json.
Returning the value itself
[2024-08-24:23:33:53:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-08-24:23:33:53:INFO] No GPUs detected (normal if

In [35]:
estimator_B = train_xgb_model(**modelB_hyperparameters)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: smsspam-xgb-2024-08-24-23-35-03-315-2024-08-24-23-35-03-319


2024-08-24 23:35:03 Starting - Starting the training job...
2024-08-24 23:35:19 Starting - Preparing the instances for training...
2024-08-24 23:35:59 Downloading - Downloading the training image.........
2024-08-24 23:37:10 Training - Training image download completed. Training in progress.[2024-08-24 23:37:17.507 ip-10-0-224-201.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-08-24 23:37:17.528 ip-10-0-224-201.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-08-24:23:37:17:INFO] Imported framework sagemaker_xgboost_container.training
[2024-08-24:23:37:17:INFO] Failed to parse hyperparameter eval_metric value logloss to Json.
Returning the value itself
[2024-08-24:23:37:17:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-08-24:23:37:17:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-24:23:37:17:INFO] Running XG

### 싱글 모델 테스트

싱글 모델 배포

In [38]:
predictor = estimator_A.deploy(initial_instance_count=1, instance_type='ml.m5.xlarge')

INFO:sagemaker:Creating model with name: smsspam-xgb-2024-08-24-23-31-19-942-2024-08-24-23-39-14-009
INFO:sagemaker:Creating endpoint-config with name smsspam-xgb-2024-08-24-23-31-19-942-2024-08-24-23-39-14-009
INFO:sagemaker:Creating endpoint with name smsspam-xgb-2024-08-24-23-31-19-942-2024-08-24-23-39-14-009


------!

In [40]:
predictor.endpoint_name

'smsspam-xgb-2024-08-24-23-31-19-942-2024-08-24-23-39-14-009'

싱글 모델 테스트

In [41]:
runtime_sm_client = boto3_session.client('sagemaker-runtime')

In [42]:
serializer = sagemaker.serializers.CSVSerializer()  
payload = serializer.serialize(X_test)
response = runtime_sm_client.invoke_endpoint(
    EndpointName=predictor.endpoint_name,
    ContentType='text/csv',
    Body=payload
)

In [43]:
predictions = [float(x) for x in response['Body'].read().decode('utf-8').strip().split('\n')]
predictions = [pred > 0.5 for pred in predictions]
predictions

[True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 Fals

성능 평가

In [44]:
# 성능 평가
print('정확도:', accuracy_score(y_test, predictions))
print('혼동 행렬:\n', confusion_matrix(y_test, predictions))
print('정밀도:', precision_score(y_test, predictions))

# 예측기 삭제 (리소스 정리)
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: smsspam-xgb-2024-08-24-23-31-19-942-2024-08-24-23-39-14-009


정확도: 0.9477756286266924
혼동 행렬:
 [[439   8]
 [ 19  51]]
정밀도: 0.864406779661017


INFO:sagemaker:Deleting endpoint with name: smsspam-xgb-2024-08-24-23-31-19-942-2024-08-24-23-39-14-009


### 멀티 모델 엔드포인트 구성
- 다음 과정은 콘솔에서 진행할수도 있습니다.
- 콘솔 진행과정은 튜토리얼을 참고하세요

멀티 모델 아티팩트를 특정 폴더 하위로 정리

In [46]:
# 멀티 모델 엔드포인트용 S3 경로 설정
s3_xgb_models_location = f"s3://{bucket_name}/{project_name}/models"
print('멀티 모델 엔드포인트용 모델 데이터 위치 : ', s3_xgb_models_location)

# 모델 A와 B의 아티팩트 경로
model_a_artifact = estimator_A.model_data
model_b_artifact = estimator_B.model_data

# 모델 A 아티팩트 복사
model_a_target_path = f"{s3_xgb_models_location}/model_a/"
wr.s3.copy_objects(
    paths=[model_a_artifact],
    source_path=model_a_artifact.replace("model.tar.gz", ''),
    target_path=model_a_target_path,
    boto3_session=boto3_session
)

# 모델 B 아티팩트 복사
model_b_target_path = f"{s3_xgb_models_location}/model_b/"
wr.s3.copy_objects(
    paths=[model_b_artifact],
    source_path=model_b_artifact.replace("model.tar.gz", ''),
    target_path=model_b_target_path,
    boto3_session=boto3_session
)

print("모델 아티팩트가 다음 위치로 복사되었습니다:")
print(f"모델 A: {model_a_target_path}")
print(f"모델 B: {model_b_target_path}")

멀티 모델 엔드포인트용 모델 데이터 위치 :  s3://dante-sagemaker/smsspam/models
모델 아티팩트가 다음 위치로 복사되었습니다:
모델 A: s3://dante-sagemaker/smsspam/models/model_a/
모델 B: s3://dante-sagemaker/smsspam/models/model_b/


In [53]:
sm_client = boto3_session.client(service_name="sagemaker")

multi_model_name = name_from_base(project_name + '-' + 'xgb-multi-model')
endpoint_name = name_from_base(project_name + '-' + 'xgb-multi-model-endpoint')
endpoint_config_name = name_from_base(project_name + '-' + 'xgb-multi-model-endpoint-config')

멀티 모델 생성

In [54]:
# 멀티 모델 생성
# ModelName: 멀티 모델의 이름
# ExecutionRoleArn: 모델 실행에 필요한 IAM 역할
# PrimaryContainer: 모델 컨테이너 설정
#   - Image: 모델 서빙에 사용할 도커 이미지 URI
#   - ModelDataUrl: 모델 아티팩트가 저장된 S3 위치
#   - Mode: "MultiModel"로 설정하여 멀티 모델 모드 활성화
create_model_response = sm_client.create_model(
    ModelName=multi_model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": estimator_A.image_uri,
        "ModelDataUrl": s3_xgb_models_location,
        "Mode": "MultiModel"
    }
)

# 생성된 멀티 모델의 ARN 출력
print("멀티 모델 ARN:", create_model_response["ModelArn"])

멀티 모델 ARN: arn:aws:sagemaker:ap-northeast-2:905418381372:model/smsspam-xgb-multi-model-2024-08-24-23-51-41-976


멀티 모델 엔드포인트 구성 생성

In [56]:
# 멀티 모델 엔드포인트 구성 생성
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,  # 엔드포인트 구성의 이름 지정
    ProductionVariants=[
        {
            "VariantName": "AllTraffic",  # 변형의 이름 (모든 트래픽을 처리)
            "ModelName": multi_model_name,  # 사용할 모델의 이름
            "InitialInstanceCount": 1,  # 초기 인스턴스 수
            "InstanceType": "ml.m5.xlarge",  # 사용할 인스턴스 유형
            "InitialVariantWeight": 1,  # 초기 변형 가중치
            "ContainerStartupHealthCheckTimeoutInSeconds": 300,  # 컨테이너 시작 상태 확인 타임아웃 시간
        }
    ],
)

print("멀티 모델 엔드포인트 구성 ARN:", endpoint_config_response["EndpointConfigArn"])


멀티 모델 엔드포인트 구성 ARN: arn:aws:sagemaker:ap-northeast-2:905418381372:endpoint-config/smsspam-xgb-multi-model-endpoint-config-2024-08-24-23-51-41-976


멀티 모델 엔드포인트 생성

In [57]:
# 멀티 모델 엔드포인트 생성
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)

print("엔드포인트 ARN:", create_endpoint_response["EndpointArn"])

# 엔드포인트 생성 완료 대기
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

print(f"멀티 모델 엔드포인트 '{endpoint_name}'가 성공적으로 생성되었습니다.")

엔드포인트 ARN: arn:aws:sagemaker:ap-northeast-2:905418381372:endpoint/smsspam-xgb-multi-model-endpoint-2024-08-24-23-51-41-976
멀티 모델 엔드포인트 'smsspam-xgb-multi-model-endpoint-2024-08-24-23-51-41-976'가 성공적으로 생성되었습니다.


### 멀티 모델 엔드포인트 테스트

모델 아티팩트 확인

In [60]:
# 모델 설명 가져오기
model_response = sm_client.describe_model(ModelName=multi_model_name)

# 모델 아티팩트 경로 출력
print(f"모델 이름: {multi_model_name}")
print(f"모델 아티팩트 경로: {model_response['PrimaryContainer']['ModelDataUrl']}")

모델 이름: smsspam-xgb-multi-model-2024-08-24-23-51-41-976
모델 아티팩트 경로: s3://dante-sagemaker/smsspam/models


In [62]:
# model.tar.gz 파일만 필터링하여 가져오기
model_files = wr.s3.list_objects(s3_xgb_models_location, suffix='.tar.gz', boto3_session=boto3_session)

# 모델 아티팩트 파일 목록 출력
print("모델 아티팩트 파일 목록:")
for file in model_files:
    print(f"- {file}")


모델 아티팩트 파일 목록:
- s3://dante-sagemaker/smsspam/models/model_a/model.tar.gz
- s3://dante-sagemaker/smsspam/models/model_b/model.tar.gz


2가지 모델에 대한 예측 수행

In [63]:
# 각 모델 아티팩트에 대해 예측 수행
predictions = {}
for model_file in model_files:
    
    model_name = model_file.replace(s3_xgb_output_location + "/", '').replace('/output/model.tar.gz', '')
    print(f'{model_name} 모델로 예측 중...')
    # 엔드포인트 호출
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='text/csv',
        Body=serializer.serialize(X_test),
        TargetModel=model_file.replace(s3_xgb_models_location, '')
    )
    
    # 예측 결과 저장
    predictions[model_name] = json.loads(response['Body'].read().decode('utf-8'))
    predictions[model_name] = [pred > 0.5 for pred in predictions[model_name]]

s3://dante-sagemaker/smsspam/models/model_a/model.tar.gz 모델로 예측 중...
s3://dante-sagemaker/smsspam/models/model_b/model.tar.gz 모델로 예측 중...


In [64]:
# 각 모델의 예측 결과 평가
for model_name, pred in predictions.items():
    print(f"\n{model_name} 모델 평가 결과:")
    print(f"정확도: {accuracy_score(y_test, pred):.4f}")
    print(f"정밀도: {precision_score(y_test, pred):.4f}")
    print(f"재현율: {recall_score(y_test, pred):.4f}")
    print(f"F1 점수: {f1_score(y_test, pred):.4f}")



s3://dante-sagemaker/smsspam/models/model_a/model.tar.gz 모델 평가 결과:
정확도: 0.9478
정밀도: 0.8644
재현율: 0.7286
F1 점수: 0.7907

s3://dante-sagemaker/smsspam/models/model_b/model.tar.gz 모델 평가 결과:
정확도: 0.9439
정밀도: 0.8475
재현율: 0.7143
F1 점수: 0.7752


리소스 삭제

In [65]:
# 엔드포인트삭제 
print("엔드포인트 삭제 중...")
sm_client.delete_endpoint(EndpointName=endpoint_name)
print(f"엔드포인트 '{endpoint_name}'이(가) 삭제되었습니다.")

# 엔드포인트 구성 삭제
print("엔드포인트 구성 삭제 중...")
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
print(f"엔드포인트 구성 '{endpoint_config_name}'이(가) 삭제되었습니다.")

# 모델 삭제
print("모델 삭제 중...")
sm_client.delete_model(ModelName=multi_model_name)
print(f"모델 '{multi_model_name}'이(가) 삭제되었습니다.")

엔드포인트 삭제 중...
엔드포인트 'smsspam-xgb-multi-model-endpoint-2024-08-24-23-51-41-976'이(가) 삭제되었습니다.
엔드포인트 구성 삭제 중...
엔드포인트 구성 'smsspam-xgb-multi-model-endpoint-config-2024-08-24-23-51-41-976'이(가) 삭제되었습니다.
모델 삭제 중...
모델 'smsspam-xgb-multi-model-2024-08-24-23-51-41-976'이(가) 삭제되었습니다.
